# Challenge 1 - Who Have Published What At Where? 

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./ironhack_service_account_big_query.json"
import six
six.moves.reload_module(six)
from google.cloud import bigquery

In [2]:
client = bigquery.Client()

In [3]:
sql="""
SELECT 
    A.au_id as AUTHOR_ID, 
    A.au_lname as LAST_NAME, 
    A.au_fname as FIRST_NAME,
    T.title as TITLE, 
    P.pub_name as PUBLISHER 
FROM
    `ironhack-data-analytics-265219.publications.authors` A 
INNER JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` TA 
ON
    A.au_id=TA.au_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.titles` T 
ON
    TA.title_id=T.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.publishers` P
ON 
    T.pub_id=P.pub_id
ORDER BY 
    AUTHOR_ID
    """

In [4]:
query_job = client.query(query=sql)
df=query_job.to_dataframe()
df.head(10)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TITLE,PUBLISHER
0,172-32-1176,White,Johnson,Prolonged Data Deprivation: Four Case Studies,New Moon Books
1,213-46-8915,Green,Marjorie,The Busy Executive's Database Guide,Algodata Infosystems
2,213-46-8915,Green,Marjorie,You Can Combat Computer Stress!,New Moon Books
3,238-95-7766,Carson,Cheryl,But Is It User Friendly?,Algodata Infosystems
4,267-41-2394,O'Leary,Michael,Cooking with Computers: Surreptitious Balance ...,Algodata Infosystems
5,267-41-2394,O'Leary,Michael,"Sushi, Anyone?",Binnet & Hardley
6,274-80-9391,Straight,Dean,Straight Talk About Computers,Algodata Infosystems
7,409-56-7008,Bennet,Abraham,The Busy Executive's Database Guide,Algodata Infosystems
8,427-17-2319,Dull,Ann,Secrets of Silicon Valley,Algodata Infosystems
9,472-27-2349,Gringlesby,Burt,"Sushi, Anyone?",Binnet & Hardley


In [9]:
sql="""
SELECT 
    au_id
FROM
    `ironhack-data-analytics-265219.publications.titleauthor`"""

In [10]:
query_job = client.query(query=sql)
df2=query_job.to_dataframe()

In [11]:
df2.head()

,au_id
0,899-46-2035
1,846-92-7186
2,213-46-8915
3,267-41-2394
4,267-41-2394


In [12]:
print(df.AUTHOR_ID.count()==df2.au_id.count())

True


# Challenge 2 - Who Have Published How Many At Where?

In [21]:
sql="""
SELECT 
    A.au_id as AUTHOR_ID, 
    A.au_lname as LAST_NAME, 
    A.au_fname as FIRST_NAME,
    P.pub_name as PUBLISHER,
    COUNT(1) as TITLE_COUNT
FROM
    `ironhack-data-analytics-265219.publications.authors` A 
INNER JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` TA 
ON
    A.au_id=TA.au_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.titles` T 
ON
    TA.title_id=T.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.publishers` P
ON 
    T.pub_id=P.pub_id
GROUP BY 
    A.au_id, 
    A.au_lname, 
    A.au_fname,
    P.pub_name
ORDER BY COUNT(1) DESC
    """

In [22]:
query_job = client.query(query=sql)
df3=query_job.to_dataframe()
df3.head(30)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,PUBLISHER,TITLE_COUNT
0,998-72-3567,Ringer,Albert,New Moon Books,2
1,807-91-6654,Panteley,Sylvia,Binnet & Hardley,1
2,722-51-5454,DeFrance,Michel,Binnet & Hardley,1
3,712-45-1867,del Castillo,Innes,Binnet & Hardley,1
4,899-46-2035,Ringer,Anne,New Moon Books,1
5,899-46-2035,Ringer,Anne,Binnet & Hardley,1
6,172-32-1176,White,Johnson,New Moon Books,1
7,486-29-1786,Locksley,Charlene,New Moon Books,1
8,486-29-1786,Locksley,Charlene,Algodata Infosystems,1
9,846-92-7186,Hunter,Sheryl,Algodata Infosystems,1


In [23]:
sql="""
SELECT 
    au_id
FROM
    `ironhack-data-analytics-265219.publications.titleauthor`"""

In [24]:
query_job = client.query(query=sql)
df4=query_job.to_dataframe()

In [25]:
df4.head()

,au_id
0,899-46-2035
1,846-92-7186
2,213-46-8915
3,267-41-2394
4,267-41-2394


In [26]:
print(df3.TITLE_COUNT.sum()==df4.au_id.count())

True


# Challenge 3 - Best Selling Authors

In [33]:
sql="""
SELECT 
    A.au_id as AUTHOR_ID, 
    A.au_lname as LAST_NAME, 
    A.au_fname as FIRST_NAME,
    IFNULL(SUM(qty),0) as TOTAL
FROM
    `ironhack-data-analytics-265219.publications.authors` A 
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` TA 
ON
    A.au_id=TA.au_id
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titles` T 
ON
    TA.title_id=T.title_id
LEFT JOIN
    `ironhack-data-analytics-265219.publications.sales` S
ON 
    T.title_id=S.title_id
GROUP BY 
    A.au_id, 
    A.au_lname, 
    A.au_fname
ORDER BY TOTAL DESC
LIMIT 3
    """

In [34]:
query_job = client.query(query=sql)
df3=query_job.to_dataframe()
df3.head(30)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TOTAL
0,899-46-2035,Ringer,Anne,148
1,998-72-3567,Ringer,Albert,133
2,846-92-7186,Hunter,Sheryl,50


# Challenge 4 - Best Selling Authors Ranking

In [35]:
sql="""
SELECT 
    A.au_id as AUTHOR_ID, 
    A.au_lname as LAST_NAME, 
    A.au_fname as FIRST_NAME,
    IFNULL(SUM(qty),0) as TOTAL
FROM
    `ironhack-data-analytics-265219.publications.authors` A 
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` TA 
ON
    A.au_id=TA.au_id
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titles` T 
ON
    TA.title_id=T.title_id
LEFT JOIN
    `ironhack-data-analytics-265219.publications.sales` S
ON 
    T.title_id=S.title_id
GROUP BY 
    A.au_id, 
    A.au_lname, 
    A.au_fname
ORDER BY TOTAL DESC
LIMIT 23
    """

In [36]:
query_job = client.query(query=sql)
df3=query_job.to_dataframe()
df3.head(30)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,TOTAL
0,899-46-2035,Ringer,Anne,148
1,998-72-3567,Ringer,Albert,133
2,846-92-7186,Hunter,Sheryl,50
3,427-17-2319,Dull,Ann,50
4,213-46-8915,Green,Marjorie,50
5,724-80-9391,MacFeather,Stearns,45
6,267-41-2394,O'Leary,Michael,45
7,807-91-6654,Panteley,Sylvia,40
8,722-51-5454,DeFrance,Michel,40
9,238-95-7766,Carson,Cheryl,30


# Bonus Challenge - Most Profiting Authors

In [210]:
sql="""
SELECT
    A.au_id as AUTHOR_ID, 
    A.au_lname as LAST_NAME, 
    A.au_fname as FIRST_NAME,
    SUM(B.ROYALTY) ROYALTY,
    SUM(B.ADVANCE) ADVANCE,
    SUM(B.ROYALTY)+SUM(B.ADVANCE) as PROFIT
FROM
    `ironhack-data-analytics-265219.publications.authors` A 
LEFT JOIN
(
    SELECT 
        TA.au_id,
        TA.title_id, 
        (IFNULL(AVG(T.price),0)*IFNULL(SUM(S.qty),0))*(AVG(T.royalty)/100)*(AVG(TA.royaltyper)/100) AS ROYALTY,
        IFNULL(AVG(T.advance),0)*(AVG(TA.royaltyper)/100) AS ADVANCE
    FROM
        `ironhack-data-analytics-265219.publications.titleauthor` TA 
    LEFT JOIN `ironhack-data-analytics-265219.publications.titles` T 
    ON
        TA.title_id=T.title_id
    LEFT JOIN
        `ironhack-data-analytics-265219.publications.sales` S
    ON 
        T.title_id=S.title_id
    GROUP BY
        TA.au_id, 
        TA.title_id
) B
ON A.au_id=B.au_id
GROUP BY 
    A.au_id,
    A.au_lname, 
    A.au_fname
ORDER BY PROFIT DESC
LIMIT 3
    """

In [211]:
query_job = client.query(query=sql)
df3=query_job.to_dataframe()
df3.head(70)

,AUTHOR_ID,LAST_NAME,FIRST_NAME,ROYALTY,ADVANCE,PROFIT
0,213-46-8915,Green,Marjorie,37.110,12125.0,12162.110
1,722-51-5454,DeFrance,Michel,21.528,11250.0,11271.528
2,998-72-3567,Ringer,Albert,88.456,7137.5,7225.956
